In [ ]:
import pandas as pd
fp = "../data/incident_event_log_02.csv"
df = pd.read_csv(fp)

In [ ]:
df_size = pd.DataFrame.from_records([{"rows": df.shape[0], "cols": df.shape[1]}])

In [ ]:
col_df = pd.DataFrame({"columns": df.columns.tolist()})

In [ ]:
df_nv = df.isnull().any().to_frame().reset_index()
df_nv.columns = ["column", "null_values"]
df_nv.to_json()

In [ ]:
df_raw_types = df.dtypes.to_frame().reset_index()
df_raw_types.columns = ["column", "dtype"]
df_raw_types["dtype"] = df_raw_types["dtype"].astype(str)
df_raw_summary = df_raw_types.merge(df_nv, on="column")
df_raw_summary

In [ ]:
cols = df.columns.tolist()
int_cols = [c for c in cols if "count" in c]
date_time_cols = [c for c in cols if "_at" in c]


In [ ]:
raw_type_info = df.dtypes.to_dict()
mod_type_info = raw_type_info.copy()

In [ ]:
import dateutil.parser as parser

def valid_time_stamp(x):
    try:
        pdt = parser.parse(x)
        return True
    except:
        return False

In [ ]:
from dask.distributed import Client
dask_client = Client()
dask_client

In [ ]:
import dask
from dask.dataframe import from_pandas
ddf = from_pandas(df, npartitions=4)

In [ ]:
valid_sys_creation = ddf["sys_created_at"].apply(valid_time_stamp, meta=('sys_created_at', str)).compute()

In [ ]:
valid_opened_at = ddf["opened_at"].apply(valid_time_stamp, meta=('opened_at', str)).compute()

In [ ]:
valid_resolved_at = ddf["resolved_at"].apply(valid_time_stamp, meta=('resolved_at', str)).compute()

In [ ]:
valid_closed_at = ddf["closed_at"].apply(valid_time_stamp, meta=('closed_at', str)).compute()

In [ ]:
valid_dates = valid_sys_creation & valid_closed_at & valid_opened_at & valid_resolved_at

In [ ]:
ddf = ddf[valid_dates].compute()

In [ ]:
ddf.shape

In [ ]:
import dask as dd
ddf["sys_created_at"] = ddf["sys_created_at"].apply(lambda x: parser.parse(x))
ddf["opened_at"] = ddf["opened_at"].apply(lambda x: parser.parse(x))
ddf["resolved_at"] = ddf["resolved_at"].apply(lambda x: parser.parse(x))
ddf["closed_at"] = ddf["closed_at"].apply(lambda x: parser.parse(x))
ddf["sys_updated_at"] = ddf["sys_updated_at"].apply(lambda x: parser.parse(x))

In [ ]:
for k,v in mod_type_info.items():
    if k in int_cols:
        mod_type_info[k] = "int64"
    if k in date_time_cols:
        mod_type_info[k] = "datetime64[ns]"
    

In [ ]:
category_cols = [ c for c in cols if (c not in int_cols and c not in date_time_cols)]

In [ ]:
category_cols

In [ ]:
nan_strings = "UNKNOWN"
for c in category_cols:
    ddf[c] = ddf[c].fillna("UNKNOWN")
    ddf[c] = ddf[c].astype(str)

In [ ]:
df_nv_fix_check = ddf.isnull().any().to_frame().reset_index()
df_nv_fix_check.columns = ["columns", "null_values"]

In [ ]:
df_nv_fix_check

In [ ]:
mod_type_info

In [ ]:
ddf.dtypes

In [ ]:
ddf_qy = ddf[(ddf["closed_at"].dt.quarter == 2) & (ddf["closed_at"].dt.year == 2016)]
fp = "../data/ticket_resolution_Q2_2016.csv"
ddf_qy.to_csv(fp, index=False)


In [ ]:
df_types = ddf.dtypes.to_frame().reset_index()
df_types.columns = ["attribute", "type"]
fp ="../data/data_types.csv"
df_types.to_csv(fp, index=False)

In [ ]:
df_types